In [0]:
import os
from typing import List, Dict
import re
from datetime import datetime as dt
import pandas as pd
import numpy as np
import boto3
from boto3.dynamodb.conditions import Key

In [0]:
!pip install janome

     |████████████████████████████████| 21.5MB 24.0MB/s 


In [0]:
!pip install mojimoji

  Created wheel for mojimoji: filename=mojimoji-0.0.9-cp36-cp36m-linux_x86_64.whl size=117560 sha256=e7601b76b9a5cc334a378547d1c746c5b73b7af4852b4843d3f8f9767eb9a3a0
  Stored in directory: /root/.cache/pip/wheels/67/f0/6e/03e5c5017afc8230e19abeacad57189138207fea2f5dd71880
Successfully built mojimoji


In [0]:
from janome.tokenizer import Tokenizer
import mojimoji

In [0]:
_aws_session = boto3.session.Session(
    region_name='ap-northeast-1',
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

DYNAMO_DB = _aws_session.resource('dynamodb')

In [0]:
class DynamoDB:

    @staticmethod
    def put_items(
        table_name: str,
        items: List[Dict],
        use_batch_writer: bool = False,
    ) -> List:
        """[summary]
        
        Args:
            table_name (str): [description]
            items (List[Dict]): [description]
        
        Raises:
            e: [description]
        
        Returns:
            List: [description]
        """
        if not isinstance(items, list):
            items = [items]

        responses = []
        try:
            table = DYNAMO_DB.Table(table_name)
        except Exception as e:
            print(f'Failed at DYNAMO_DB.Table(table_name) : {e}')
            raise e

        if use_batch_writer:
            with table.batch_writer() as batch:
                for item in items:
                    try:
                        responses.append(
                            batch.put_item(
                                Item=item,
                            )
                        )
                    except Exception as e:
                        print(f'Failed to put data to DynamoDB. Skipping : {e}')
        else:
            for item in items:
                try:
                    responses.append(
                        table.put_item(
                            TableName=table_name,
                            Item=item,
                        )
                    )
                except Exception as e:
                    print(f'Failed to put data to DynamoDB. Skipping : {e}')

        return responses

    @staticmethod
    def partitionkey_query(
        table_name: str,
        partition_key_name: str,
        partition_key: str,
    ) -> List[Dict]:
        try:
            table = DYNAMO_DB.Table(table_name)
        except Exception as e:
            print(f'Failed at DYNAMO_DB.Table(table_name) : {e}')
            raise e

        try:
            response = table.query(
                KeyConditionExpression=Key(partition_key_name).eq(partition_key)
            )
        except Exception as e:
            print(f'Failed to query : {e}')
            return []

        return response['Items']

In [0]:
res = DynamoDB.partitionkey_query(
    table_name='finapp_twitter_tweet',
    partition_key_name='keyword',
    partition_key='ソレキア',
)

In [0]:
res[:2]

[{'contributors': None,
  'coordinates': None,
  'created_at': Decimal('1575920641'),
  'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
  'favorite_count': Decimal('0'),
  'favorited': False,
  'geo': None,
  'id': Decimal('1204260456770072578'),
  'id_str': '1204260456770072578',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'keyword': 'ソレキア',
  'lang': 'ja',
  'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'},
  'place': None,
  'retweet_count': Decimal('0'),
  'retweeted': False,
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'text': '５０％以上持ってるソレキアとか明らかに調子いいのに株価死んでたんだからためらいなく買っとけばよかった\n決算よかったのに下げて混乱してしまった',
  'truncated': False,
  'user': {'contributors_enabled': False,
   'created_at': 'Sat Jul 10 21:33:21 +0000 2010',
   'default_profile': Fal

In [0]:
len(res)

293

In [0]:
[dt.fromtimestamp(r['created_at']) for r in res]

[datetime.datetime(2019, 12, 9, 19, 44, 1),
 datetime.datetime(2019, 12, 10, 3, 10, 15),
 datetime.datetime(2019, 12, 10, 3, 17, 24),
 datetime.datetime(2019, 12, 12, 17, 5, 9),
 datetime.datetime(2019, 12, 14, 3, 45, 35),
 datetime.datetime(2019, 12, 22, 18, 19, 40),
 datetime.datetime(2019, 12, 22, 18, 48, 41),
 datetime.datetime(2019, 12, 22, 18, 50, 51),
 datetime.datetime(2019, 12, 22, 19, 19, 20),
 datetime.datetime(2019, 12, 22, 20, 7, 19),
 datetime.datetime(2019, 12, 22, 20, 46, 32),
 datetime.datetime(2019, 12, 22, 21, 5, 47),
 datetime.datetime(2019, 12, 22, 21, 5, 58),
 datetime.datetime(2019, 12, 22, 21, 6, 11),
 datetime.datetime(2019, 12, 22, 21, 6, 14),
 datetime.datetime(2019, 12, 22, 21, 6, 31),
 datetime.datetime(2019, 12, 22, 21, 6, 46),
 datetime.datetime(2019, 12, 22, 21, 6, 49),
 datetime.datetime(2019, 12, 22, 21, 8, 51),
 datetime.datetime(2019, 12, 22, 21, 9, 19),
 datetime.datetime(2019, 12, 22, 21, 9, 59),
 datetime.datetime(2019, 12, 22, 21, 11, 22),
 datet

In [0]:
len([r['text'] for r in res])

293

In [0]:
len(set([r['text'] for r in res]))

179

In [0]:
[(int(r['user']['followers_count']), r['text']) for r in res]

[(308,
  '５０％以上持ってるソレキアとか明らかに調子いいのに株価死んでたんだからためらいなく買っとけばよかった\n決算よかったのに下げて混乱してしまった'),
 (989,
  'マクロスが上がった訳がわかってきた\n\nこれを見て欲しいです。\n\nマクロスは投資会社でもあって\n\n技研HLD\nソレキア\n\nの筆頭株主です。\nそれぞれ業績が絶好調で\n\n技研HLDは上方修正期待\nソレキアが減益予想がまさかの前期比増益の… https://t.co/zuTKoK7A8c'),
 (989,
  '含み益だけで\n\nソレキア11月比\n24万株×1000円上昇＝2.4億\n\n技研HLD11月比\n430万株×150円上昇＝6.45億\n\n合計8.85億も利益が増える。\n\n中間決算の当期利益は2.42億。\n通期予想は4.2億。\n\nつまり… https://t.co/WdeSJOKscH'),
 (3641, '第二のソレキアになるか'),
 (117, '@kanetenn 2173博展  53億\n9867ソレキア36億\nお願いします(^^)'),
 (69, '9867 ソレキアです。\n前場で少し買いました。\n\nまさか今日来るとは。。\n想定外'),
 (815,
  '9867 ソレキア\n\nストップになる前にちょっと買えたので、ストップ高で売っちゃいます。\n小銭稼ぎ。\nまだ上がりそうな割安ですが、利益率低いのが気になるのでやめときます。 https://t.co/Hl8jzwow4H'),
 (815,
  '6343 フリージアマクロス\n\nソレキアはフリージアマクロスの持分法適用なので少し買っておきます。 https://t.co/i1NXC75xVi'),
 (11509, '大谷工業の次はソレキアなんですかね\nこういう銘柄を仕掛ける人も凄いけど買う人も凄いですよね'),
 (577,
  'この前は夢みつけ隊が噴いたら、つられてフリージア・マクロスも噴いたけど\n\nソレキア噴いても、フリージア・マクロス噴かないのは謎\n\nてことで、1万株だけin'),
 (799, '12月23日のストップ高銘柄\n【9867】ソレキア\n（東証JQS）\nhttps://t.co/iETCnMav84'),


In [0]:
j_t = Tokenizer()

In [0]:
def preprocess_text(text):
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    text = mojimoji.zen_to_han(text, kana=False)

    return text

In [0]:
for text in [r['text'] for r in res]:
    print(j_t.tokenize(preprocess_text(text), wakati=True))

['50', '%', '以上', '持っ', 'てる', 'ソレキア', 'とか', '明らか', 'に', '調子', 'いい', 'のに', '株価', '死ん', 'で', 'たん', 'だ', 'から', 'ためらい', 'なく', '買っ', 'とけ', 'ば', 'よかっ', 'た', '決算', 'よかっ', 'た', 'の', 'に', '下げ', 'て', '混乱', 'し', 'て', 'しまっ', 'た']
['マクロス', 'が', '上がっ', 'た', '訳', 'が', 'わかっ', 'て', 'き', 'た', 'これ', 'を', '見', 'て', '欲しい', 'です', '。', 'マクロス', 'は', '投資', '会社', 'でも', 'あっ', 'て', '技研', 'HLD', 'ソレキア', 'の', '筆頭', '株主', 'です', '。', 'それぞれ', '業績', 'が', '絶好調', 'で', '技研', 'HLD', 'は', '上方', '修正', '期待', 'ソレキア', 'が', '減益', '予想', 'が', 'まさかの', '前期', '比', '増益', 'の', '…']
['含み益', 'だけ', 'で', 'ソレキア', '11', '月', '比', '24', '万', '株', '×', '1000', '円', '上昇', '=', '2', '.', '4', '億', '技研', 'HLD', '11', '月', '比', '430', '万', '株', '×', '150', '円', '上昇', '=', '6', '.', '45', '億', '合計', '8', '.', '85', '億', 'も', '利益', 'が', '増える', '。', '中間', '決算', 'の', '当期', '利益', 'は', '2', '.', '42', '億', '。', '通期', '予想', 'は', '4', '.', '2', '億', '。', 'つまり', '…']
['第', '二', 'の', 'ソレキア', 'に', 'なる', 'か']
['@', 'kanetenn', '2173', '博', '展', '53', '億', '98

In [0]:
!sudo apt install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
curl is already the newest version (7.58.0-2ubuntu3.8).
git is already the newest version (1:2.17.1-1ubuntu0.5).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils swig
  swig3.0
0 upgraded, 13 newly installed, 0 to remove an

In [0]:
!git clone https://github.com/neologd/mecab-ipadic-neologd.git

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 9113, done.
remote: Total 9113 (delta 0), reused 0 (delta 0), pack-reused 9113
Receiving objects: 100% (9113/9113), 412.95 MiB | 37.48 MiB/s, done.
Resolving deltas: 100% (5248/5248), done.


In [0]:
%cd mecab-ipadic-neologd

/content/mecab-ipadic-neologd


In [0]:
!sudo bin/install-mecab-ipadic-neologd

[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEologd] :     iconv => ok
[install-mecab-ipadic-NEologd] :     patch => ok
[install-mecab-ipadi

In [0]:
%cd ..

/content


In [0]:
!pip install mecab-python3

     |████████████████████████████████| 17.1MB 242kB/s 


In [0]:
import MeCab

In [0]:
m_t = MeCab.Tagger('-Owakati -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

In [0]:
for text in set([r['text'] for r in res]):
    print(m_t.parse(preprocess_text(text)).strip().split())

['RT', '@', 'JawsKabu', ':', '2019', '/', '12', '/', '26', 'ｽﾄｯﾌﾟ', '高', '7', '社', '*', '理由', 'は', '添付', '1枚', '､', 'ﾃﾞｰﾀ', 'は', '以下', 'ﾘﾝｸ', '･', 'HD', '3750', '…']
['ｿﾚｷｱ', 'やばい', 'な!', '全然', '乗れ', 'て', 'ない', '｡']
['こっち', 'の', '上位', '狙っ', 'て', 'ｿﾚｷｱ', '出し', 'た', 'のに', 'な', 'ｰ', '🙄', '指値', 'じゃなくて', '上', 'の', '板', '買っ', 'て', 'おく', 'べき', 'だっ', 'た', '😭', '😭']
['@', 'takosensei', '2019', 'ｿﾚｷｱ', 'で', 'お願い', 'し', 'ます', 'w']
['続', ')', 'S', '高', '9867', 'ｿﾚｷｱ', '4', '連', 'S', '高', '※', '74億円', '好', '決算', '期待', '続く', '9878', 'ｾｷﾄﾞ', '※', '11億円', '中国', '新幹線', '教育', 'ｸﾞﾙｰﾌﾟ', 'と', 'の', '合弁会社', 'を', '来年1月', 'に', '設立', 'へ']
['値上がり', '1位', '9867', 'ｿﾚｷｱ', '21', '.', '0%', '2位', '9878', 'ｾｷﾄﾞ', '18.1%', '3位', '6656', 'ｲﾝｽﾍﾟｯｸ', '17.3%', '4位', '3907', 'ｼﾘｺﾝｽﾀｼﾞｵ', '16', '.', '0%', '5位', '1789', 'ETSHD', '15.9%', '6位', '3750', '…']
['まあ', 'で', 'も', 'ｿﾚｷｱ', 'は', 'pts', 'で', '売り', 'たい人', 'が', '売っ', 'て', '需給', 'が', '良', '化', 'すれ', 'ば', '良い', '訳', 'だ', 'から', '､', '良い', 'こと', 'か', '｡']
['ｽﾄｯﾌﾟ', '高', '**

In [0]:
!ls /usr/lib/mecab

mecab-cost-train  mecab-dict-index   mecab-test-gen
mecab-dict-gen	  mecab-system-eval


In [0]:
!ls /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

char.bin  left-id.def  pos-id.def   right-id.def  unk.dic
dicrc	  matrix.bin   rewrite.def  sys.dic


In [0]:
res = DynamoDB.partitionkey_query(
    table_name='finapp_twitter_tweet',
    partition_key_name='keyword',
    partition_key='大谷工業',
)

In [0]:
len(res)

340

In [0]:
[dt.fromtimestamp(r['created_at']) for r in res]

[datetime.datetime(2019, 12, 8, 14, 54, 6),
 datetime.datetime(2019, 12, 8, 14, 56, 27),
 datetime.datetime(2019, 12, 8, 14, 57, 7),
 datetime.datetime(2019, 12, 8, 15, 27, 46),
 datetime.datetime(2019, 12, 8, 16, 43, 6),
 datetime.datetime(2019, 12, 8, 17, 48, 9),
 datetime.datetime(2019, 12, 8, 18, 15, 16),
 datetime.datetime(2019, 12, 8, 19, 19, 17),
 datetime.datetime(2019, 12, 8, 19, 19, 31),
 datetime.datetime(2019, 12, 8, 19, 20),
 datetime.datetime(2019, 12, 8, 19, 21, 10),
 datetime.datetime(2019, 12, 8, 19, 21, 17),
 datetime.datetime(2019, 12, 8, 19, 21, 19),
 datetime.datetime(2019, 12, 8, 19, 30, 3),
 datetime.datetime(2019, 12, 8, 21, 23, 57),
 datetime.datetime(2019, 12, 8, 23, 6, 38),
 datetime.datetime(2019, 12, 8, 23, 35, 51),
 datetime.datetime(2019, 12, 9, 1, 30, 29),
 datetime.datetime(2019, 12, 9, 2, 29, 34),
 datetime.datetime(2019, 12, 9, 2, 30, 7),
 datetime.datetime(2019, 12, 9, 5, 31, 36),
 datetime.datetime(2019, 12, 9, 6, 0, 20),
 datetime.datetime(2019, 12

In [0]:
[(int(r['user']['followers_count']), r['text']) for r in res]

[(6084,
  '大和重、アイル、大谷工業、みらいWKS、ラアトレ、S高気配。\n\n大盛工、インスペック、山九、オオバ、S高手前。\n\n他GU気配多数。'),
 (134,
  'RT @bookpine: 大和重、アイル、大谷工業、みらいWKS、ラアトレ、S高気配。\n\n大盛工、インスペック、山九、オオバ、S高手前。\n\n他GU気配多数。'),
 (74,
  'RT @bookpine: 大和重、アイル、大谷工業、みらいWKS、ラアトレ、S高気配。\n\n大盛工、インスペック、山九、オオバ、S高手前。\n\n他GU気配多数。'),
 (18470, '大谷工業、寄らずのストップ高'),
 (540, '那須は2倍になって大谷工業も今頃鉄塔っぽい🤔 https://t.co/xN4S9zaDCH'),
 (18,
  '金曜の先出し分〜\n\nオルトは下がったけどあやしいね笑\n大谷工業ストップ高おめー🎉\n\n他も予想的ちゅー\n#大谷工業\n#鳥貴族\n#アイケイ\n#HEROZ\n#オルトプラス https://t.co/92k7Wcjj0a'),
 (900, '大谷工業板すごいなw\n\nストップ安まで27枚しか買い板ない https://t.co/aDgNnOI2Uv'),
 (38440, '【5939】大谷工業  6,050円 （+19.8%）\n（東証JQスタンダード 12/09 13:18）#株'),
 (39895, '【5939】大谷工業 6,050円\n（新高値 12/09 13:18）'),
 (60879, '大谷工業寄り\n\nこれも何がなんだか…(;´･ω･)'),
 (36363, '大谷工業\u30002日連続S高\u3000+20％ https://t.co/3hqRoUATGa'),
 (74,
  'RT @Up10pct_bot: 【5939】大谷工業  6,050円 （+19.8%）\n（東証JQスタンダード 12/09 13:18）#株'),
 (1862, 'RT @sak_07_: 大谷工業\u30002日連続S高\u3000+20％ https://t.co/3hqRoUATGa'),
 (14331,
  '【市況アラート】3854「アイル」が株トレーダーで話題に［12/9 13:30］\n\n

In [0]:
res = DynamoDB.partitionkey_query(
    table_name='finapp_twitter_tweet',
    partition_key_name='keyword',
    partition_key='セキド',
)

In [0]:
[dt.fromtimestamp(r['created_at']) for r in res]

[datetime.datetime(2019, 12, 2, 15, 59, 33),
 datetime.datetime(2019, 12, 2, 17, 21, 48),
 datetime.datetime(2019, 12, 2, 17, 44, 6),
 datetime.datetime(2019, 12, 2, 17, 45, 8),
 datetime.datetime(2019, 12, 2, 17, 54, 34),
 datetime.datetime(2019, 12, 2, 17, 54, 43),
 datetime.datetime(2019, 12, 2, 18, 17, 9),
 datetime.datetime(2019, 12, 2, 18, 29, 53),
 datetime.datetime(2019, 12, 2, 18, 46, 16),
 datetime.datetime(2019, 12, 2, 18, 50, 34),
 datetime.datetime(2019, 12, 2, 19, 0, 20),
 datetime.datetime(2019, 12, 2, 19, 3, 20),
 datetime.datetime(2019, 12, 2, 19, 8, 40),
 datetime.datetime(2019, 12, 2, 19, 28, 30),
 datetime.datetime(2019, 12, 2, 22, 52, 10),
 datetime.datetime(2019, 12, 3, 4, 17, 5),
 datetime.datetime(2019, 12, 3, 5, 7, 6),
 datetime.datetime(2019, 12, 3, 5, 8, 37),
 datetime.datetime(2019, 12, 3, 7, 14, 59),
 datetime.datetime(2019, 12, 3, 11, 27, 4),
 datetime.datetime(2019, 12, 3, 16, 57, 31),
 datetime.datetime(2019, 12, 4, 0, 7, 3),
 datetime.datetime(2019, 12,

In [0]:
[(int(r['user']['followers_count']), dt.fromtimestamp(r['created_at']), r['text']) for r in res]

[(721,
  datetime.datetime(2019, 12, 2, 15, 59, 33),
  '【新着ニュース】\n株式会社セキドは、従来のROVの基本性能に加え海底地形把握機能を搭載した最新ROVを発表しました。\nhttps://t.co/vjVRQ8VJL6'),
 (2354,
  datetime.datetime(2019, 12, 2, 17, 21, 48),
  'こんにちは(*´ω`*)\nセキド横浜ベイサイド店です！\n\n横浜ベイサイド店に #DJI #MavicMini のアクセサリーが入荷しました♪\nケースやプロペラガードはもちろん、遊びゴコロあふれるクリエイティブキットも到着です。\n担… https://t.co/dvUORGhAWi'),
 (27258,
  datetime.datetime(2019, 12, 2, 17, 44, 6),
  'セキド、海底地形把握機能を搭載した「サイドスキャンソナー搭載水中ROV」発売 https://t.co/JlFHqvvjiV'),
 (734,
  datetime.datetime(2019, 12, 2, 17, 45, 8),
  'RT @fabcrossjp: セキド、海底地形把握機能を搭載した「サイドスキャンソナー搭載水中ROV」発売 https://t.co/JlFHqvvjiV'),
 (136,
  datetime.datetime(2019, 12, 2, 17, 54, 34),
  'RT @fabcrossjp: セキド、海底地形把握機能を搭載した「サイドスキャンソナー搭載水中ROV」発売 https://t.co/JlFHqvvjiV'),
 (433,
  datetime.datetime(2019, 12, 2, 17, 54, 43),
  'RT @fabcrossjp: セキド、海底地形把握機能を搭載した「サイドスキャンソナー搭載水中ROV」発売 https://t.co/JlFHqvvjiV'),
 (141,
  datetime.datetime(2019, 12, 2, 18, 17, 9),
  'RT @fabcrossjp: セキド、海底地形把握機能を搭載した「サイドスキャンソナー搭載水中ROV」発売 http

In [0]:
df_stocklist = pd.read_csv('s3://fin-app/stocklist/stocklist.csv')

In [0]:
df_stocklist

,銘柄コード,銘柄名,市場名,業種分類,単元株数,日経225採用銘柄
0,1301,(株)極洋,東証1部,水産・農林業,100,NaN
1,1332,日本水産(株),東証1部,水産・農林業,100,1.0
2,1333,マルハニチロ(株),東証1部,水産・農林業,100,1.0
3,1352,(株)ホウスイ,東証1部,卸売業,100,NaN
4,1376,カネコ種苗(株),東証1部,水産・農林業,100,NaN
...,...,...,...,...,...,...
3861,9993,(株)ヤマザワ,東証1部,小売業,100,NaN
3862,9994,(株)やまや,東証1部,小売業,100,NaN
3863,9995,(株)グローセル,東証1部,卸売業,100,NaN
3864,9996,(株)サトー商会,東証JQS,卸売業,100,NaN


In [0]:
df_stocklist['業種分類'].value_counts()

情報・通信       475
サービス業       473
小売業         363
卸売業         334
電気機器        248
機械          232
化学          213
建設業         171
不動産業        138
食料品         126
その他製品       112
輸送用機器        95
金属製品         94
銀行業          88
REIT銘柄一覧     81
医薬品          71
陸運業          66
ガラス・土石製品     58
繊維製品         55
精密機器         52
鉄鋼           45
証券業          40
倉庫・運輸関連業     38
その他金融業       35
非鉄金属         34
パルプ・紙        26
電気・ガス業       24
ゴム製品         19
保険業          14
海運業          13
石油・石炭製品      11
水産・農林業       11
鉱業            6
空運業           5
Name: 業種分類, dtype: int64

In [0]:
df_stocklist[df_stocklist['日経225採用銘柄'] == 1]

,銘柄コード,銘柄名,市場名,業種分類,単元株数,日経225採用銘柄
1,1332,日本水産(株),東証1部,水産・農林業,100,1.0
2,1333,マルハニチロ(株),東証1部,水産・農林業,100,1.0
41,1605,国際石油開発帝石(株),東証1部,鉱業,100,1.0
51,1721,コムシスホールディングス(株),東証1部,建設業,100,1.0
86,1801,大成建設(株),東証1部,建設業,100,1.0
...,...,...,...,...,...,...
3666,9613,(株)ＮＴＴデータ,東証1部,情報・通信,100,1.0
3733,9735,セコム(株),東証1部,サービス業,100,1.0
3750,9766,コナミホールディングス(株),東証1部,情報・通信,100,1.0
3854,9983,(株)ファーストリテイリング,東証1部,小売業,100,1.0


In [0]:
res = DynamoDB.partitionkey_query(
    table_name='finapp_twitter_tweet',
    partition_key_name='keyword',
    partition_key='ソレキア',
)

In [0]:
texts = [r['text'] for r in res]
texts = [preprocess_text(text) for text in set(texts)]

In [0]:
texts

['RT@JawsKabu:2019/12/26ストップ高7社*理由は添付1枚、データは以下リンク・HD3750…',
 'ソレキアやばいな!全然乗れてない。',
 'こっちの上位狙ってソレキア出したのになー🙄指値じゃなくて上の板買っておくべきだった😭😭',
 '@takosensei2019ソレキアでお願いしますw',
 '続)S高9867ソレキア4連S高※74億円好決算期待続く9878セキド※11億円中国新幹線教育グループとの合弁会社を来年1月に設立へ',
 '値上がり1位9867ソレキア21.0%2位9878セキド18.1%3位6656インスペック17.3%4位3907シリコンスタジオ16.0%5位1789ETSHD15.9%6位3750…',
 'まあでもソレキアはptsで売りたい人が売って需給が良化すれば良い訳だから、良いことか。',
 'ストップ高**引け**3622ネットイヤーG5939大谷工業6196ストライク6387サムコ9867ソレキア**場中**3490アズ企画設計3639ボルテージ4361川口化学工業477…',
 'ストップ高銘柄13銘柄1675パラジETF1773YTL3842ネクストジェン4366ダイトーケミ4479マクアケ6380オリチエン6567SERIOHD7602カーチスHD7875竹田印刷8165千趣会…',
 '12/23の値幅大↑トレンド銘柄⤴️1773YTL⤴️1966高田工業⤴️2970グッドライフC⤴️4251恵和⤴️4478フリー⤴️7063エードット⤴️7602カーチスHD⤴️9867ソレキア',
 '続)S高6578エヌリンクス一時※30億円光通信の保有割合が上昇し需給思惑働く8251パルコ配分※1702億円JフロントがTOBで完全子会社化8257山陽百一時※20億円9263ビジョナリー…',
 '今日マクロスが年初来高値を更新した理由がわかった。筆頭株主にマクロスがある、ソレキアが5年来高値を更新しているからだ。理由はわからないが5連続ぐらいストップ高になっている。マクロスとベジさんの有価証券評価益がうなぎ登りな…',
 '大谷工業、ソレキアと続く銘柄の初動らしきものがいくつか見受けられるんだが…夢があるから乗ってみたいとは思う',
 '12月26日(木)S高9867ソレキア74億4連S好決算1789ETSH

In [0]:
len(texts)

179

In [0]:
df_train = pd.DataFrame({
    'text': texts[:130]
})
df_test = pd.DataFrame({
    'text': texts[130:]
})

In [0]:
df_train.to_csv('text_train.csv', index=None, header=None)
df_test.to_csv('text_test.csv', index=None, header=None)

In [0]:
ls

 fasttext_model.zip
 mecab-ipadic-neologd/
 model.vec
 sample_data/
 text_test.csv
 text_train.csv
'uc?export=download&confirm=-nIX&id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc'
'view?usp=sharing'


In [0]:
import torchtext

In [0]:
def mecab_tokenizer(text):
    return m_t.parse(preprocess_text(text)).strip().split()

In [0]:
max_length = 100
TEXT = torchtext.data.Field(
    sequential=True,
    tokenize=mecab_tokenizer,
    use_vocab=True,
    lower=True,
    include_lengths=True,
    batch_first=True,
    fix_length=max_length
)

In [0]:
LABEL = torchtext.data.Field(
    sequential=False,
    use_vocab=False
)

In [0]:
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='.',
    train='text_train.csv',
    validation='text_test.csv',
    test='text_test.csv',
    format='csv',
    fields=[('Text', TEXT), ('Label', LABEL)]
)

In [0]:
vars(train_ds[0])

{'Text': ['rt',
  '@',
  'jawskabu',
  ':',
  '2019',
  '/',
  '12',
  '/',
  '26',
  'ストップ高',
  '7',
  '社',
  '*',
  '理由',
  'は',
  '添付',
  '1枚',
  '、',
  'データ',
  'は',
  '以下',
  'リンク',
  '・',
  'hd',
  '3750',
  '…']}

In [0]:
len(train_ds)

130

In [0]:
TEXT.build_vocab(train_ds, min_freq=1)

In [0]:
TEXT.vocab.freqs

Counter({'rt': 14,
         '@': 18,
         'jawskabu': 1,
         ':': 15,
         '2019': 12,
         '/': 35,
         '12': 24,
         '26': 8,
         'ストップ高': 33,
         '7': 9,
         '社': 3,
         '*': 2,
         '理由': 4,
         'は': 81,
         '添付': 2,
         '1枚': 2,
         '、': 86,
         'データ': 5,
         '以下': 2,
         'リンク': 2,
         '・': 18,
         'hd': 18,
         '3750': 11,
         '…': 56,
         'ソレキア': 115,
         'やばい': 2,
         'な!': 1,
         '全然': 1,
         '乗れ': 1,
         'て': 29,
         'ない': 10,
         '。': 67,
         'こっち': 1,
         'の': 109,
         '上位': 5,
         '狙っ': 1,
         '出し': 1,
         'た': 32,
         'のに': 3,
         'なー': 1,
         '🙄': 1,
         '指値': 1,
         'じゃなくて': 1,
         '上': 1,
         '板': 1,
         '買っ': 3,
         'おく': 2,
         'べき': 2,
         'だっ': 4,
         '😭': 4,
         'takosensei': 1,
         'で': 43,
         'お願い': 2,
         'し'

In [0]:
!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc" > /dev/null
!awk '/_warning_/ {print $NF}' /tmp/cookie

-nIX


In [0]:
!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=-nIX&id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc" -o fasttext_model.zip 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   383    0   383    0     0    834      0 --:--:-- --:--:-- --:--:--   832
100  331M    0  331M    0     0   156M      0 --:--:--  0:00:02 --:--:--  235M


In [0]:
ls -l -h

total 663M
-rw-r--r--  1 root root 332M Dec 30 17:58  fasttext_model.zip
drwxr-xr-x 10 root root 4.0K Dec 30 17:19  mecab-ipadic-neologd/
drwxr-xr-x  1 root root 4.0K Dec 18 16:52  sample_data/
-rw-r--r--  1 root root 8.5K Dec 30 17:28  text_test.csv
-rw-r--r--  1 root root  25K Dec 30 17:28  text_train.csv
-rw-r--r--  1 root root 332M Dec 30 17:57 'uc?export=download&confirm=-nIX&id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc'
-rw-r--r--  1 root root  67K Dec 30 17:50 'view?usp=sharing'


In [0]:
!unzip fasttext_model.zip

Archive:  fasttext_model.zip
  inflating: model.vec               


In [0]:
ls

 fasttext_model.zip
 mecab-ipadic-neologd/
 model.vec
 sample_data/
 text_test.csv
 text_train.csv
'uc?export=download&confirm=-nIX&id=0ByFQ96A4DgSPUm9wVWRLdm5qbmc'
'view?usp=sharing'


In [0]:
from torchtext.vocab import Vectors

In [0]:
fasttext_vecs = Vectors(name='./model.vec')

100%|█████████▉| 350843/351122 [00:41<00:00, 8658.89it/s]

In [0]:
fasttext_vecs.dim

300

In [0]:
len(fasttext_vecs.itos)

351122

In [0]:
TEXT.build_vocab(train_ds, vectors=fasttext_vecs, min_freq=1)

In [0]:
TEXT.vocab.vectors.shape

torch.Size([1219, 300])

In [0]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             'ソレキア': 2,
             'の': 3,
             '、': 4,
             'は': 5,
             's': 6,
             '。': 7,
             'が': 8,
             '9867': 9,
             '…': 10,
             '銘柄': 11,
             '高': 12,
             'で': 13,
             '(': 14,
             'に': 15,
             '/': 16,
             'を': 17,
             'ストップ高': 18,
             'た': 19,
             '.': 20,
             'と': 21,
             'て': 22,
             '&': 23,
             '2': 24,
             ')': 25,
             '12': 26,
             'も': 27,
             '#': 28,
             '【': 29,
             '】': 30,
             'し': 31,
             'インスペック': 32,
             '千趣会': 33,
             '@': 34,
             'hd': 35,
             '※': 36,
             '・': 37,
             '豊商事': 38,
             '️': 39,
             '8747': 40,
             '='

In [0]:
TEXT.vocab.freqs

Counter({'rt': 14,
         '@': 18,
         'jawskabu': 1,
         ':': 15,
         '2019': 12,
         '/': 35,
         '12': 24,
         '26': 8,
         'ストップ高': 33,
         '7': 9,
         '社': 3,
         '*': 2,
         '理由': 4,
         'は': 81,
         '添付': 2,
         '1枚': 2,
         '、': 86,
         'データ': 5,
         '以下': 2,
         'リンク': 2,
         '・': 18,
         'hd': 18,
         '3750': 11,
         '…': 56,
         'ソレキア': 115,
         'やばい': 2,
         'な!': 1,
         '全然': 1,
         '乗れ': 1,
         'て': 29,
         'ない': 10,
         '。': 67,
         'こっち': 1,
         'の': 109,
         '上位': 5,
         '狙っ': 1,
         '出し': 1,
         'た': 32,
         'のに': 3,
         'なー': 1,
         '🙄': 1,
         '指値': 1,
         'じゃなくて': 1,
         '上': 1,
         '板': 1,
         '買っ': 3,
         'おく': 2,
         'べき': 2,
         'だっ': 4,
         '😭': 4,
         'takosensei': 1,
         'で': 43,
         'お願い': 2,
         'し'

In [0]:
TEXT.vocab.vectors[TEXT.vocab.stoi['ストップ高']]

tensor([-0.0927,  0.1112, -0.0735,  0.0070, -0.1556,  0.3117,  0.2195, -0.0462,
         0.1982,  0.0048,  0.1420, -0.6085, -0.3388,  0.0589,  0.0402, -0.2716,
        -0.7126,  0.1320,  0.3180, -0.2324,  0.1991, -0.0052,  0.0722, -0.3970,
         0.1958, -0.2626,  0.3562,  0.0998, -0.1320,  0.0837,  0.2504, -0.1071,
         0.2321,  0.4008,  0.1206, -0.0422,  0.4046,  0.1262, -0.2986,  0.1780,
        -0.1231, -0.2211, -0.4726, -0.1489, -0.1390,  0.1653, -0.1288, -0.1978,
        -0.0243, -0.1848,  0.2797,  0.0600, -0.1305, -0.1447,  0.1319, -0.4121,
         0.0325, -0.0256,  0.1290, -0.0401,  0.0119,  0.1947, -0.1156, -0.0605,
         0.1377,  0.2578, -0.2264,  0.1845,  0.1868, -0.2390, -0.4481, -0.2592,
        -0.1080, -0.0293,  0.0461,  0.0442,  0.3134,  0.5013,  0.1746, -0.4048,
        -0.3852, -0.1991,  0.3514, -0.0424, -0.0243,  0.1714,  0.5365, -0.0801,
         0.1090, -0.0401, -0.0766, -0.4673, -0.2586,  0.4309,  0.2824,  0.1634,
        -0.1420,  0.1268, -0.4209, -0.15

In [0]:
TEXT.vocab.vectors[TEXT.vocab.stoi['😭']]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 